In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import argparse
from glob import glob
# rdir = 'results/regression/'
# rdir = 'results_r1/'
rdir = '../../results_long/'
frames = []
# load regression results
for f in glob(rdir + '*/*R.csv'):
        
    df = pd.read_csv(f,sep='\t',index_col=None)

#     print(f)
    if 'FeatJustCorr' in f:
        continue
    alg = df.algorithm.unique()
    if len(alg)>0:
        if 'XGB' in df['algorithm'][0]: 
            df['method'] = 'XGB'
        else:
            df['method'] = df['algorithm']
    df['median_score'] = df['score'].median()
#     df['method'] = f.split('Feat')[1].split('R.csv')[0]
    df['dataset'] = f.split('/')[-2]
    
    frames.append(df)
df_reg = pd.concat(frames)


# methods =['LexNSGA2','Lex','NSGA2','SimAnneal','MLP','ElasticNet',  'Random', 'RF', 'KernelRidge']
# df_reg = df_reg.loc[df_reg.method.isin(methods)]


print(len(df_reg), 'results')

print(df_reg.columns)
print(len(df_reg['dataset'].unique()),'datasets')
print('methods:',df_reg.method.unique())
print('unique seeds:',len(df_reg.seed.unique()))
print('dataset results',df_reg.groupby(['method']).count().reset_index())
# look at summaries
counts={}
methods = df_reg.method.unique()
for d,df_d in df_reg.groupby('dataset'):
    counts[d] = len(df_d['algorithm'].unique())
cool_ds = [d for d in counts.keys() if counts[d]>=len(methods)]
notcool_ds = [d for d in counts.keys() if counts[d]<len(methods)]
print(len(cool_ds),'cool ds')
# filter datasets without results
# df_reg = df_reg.loc[df_reg['dataset'].isin(cool_ds)]
print('not cool ds:',notcool_ds)

print('methods:',df_reg.method.unique())
print('methods:',df_reg.algorithm.unique())
df_reg['score'] = df_reg['score'].astype(float)
df_reg['size'] = df_reg['size'].astype(float)

# Order = ['Feat','FeatCorr','FeatCN','XGB','RF','MLP','KernelRidge','ElasticNet']
# Order = ['Feat','XGB','RF','MLP','KernelRidge','ElasticNet']
Order = ['Feat','FeatResXO', 'FeatStageXO']

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# plt.figure(figsize=(20,5))
df_reg['norm_score'] = df_reg.groupby('dataset')['score'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df_sum=pd.DataFrame()#columns=['dataset','method','med_score'])
df_sum['med_score'] = df_reg.groupby(['dataset','method'])['score'].median().drop_duplicates()
df_sum['norm_score'] = df_reg.groupby(['dataset','method'])['norm_score'].median().drop_duplicates()

df_sum['count'] = df_reg.groupby(['dataset','method'])['score'].count()
# print(df_sum[:10])
df_sum=df_sum.reset_index(level=['dataset', 'method'])
# df_sum=df_sum.unstack(level=-1)
# print(df_sum)
# plt.figure()
# sns.barplot(data=df_sum,y='count',x='dataset',hue='method')
# plt.show()
# sns.boxplot(data=df_sum,x='method')
# sns.boxplot(data=df_sum,y='med_score',x='method',notch=True,showfliers=False, 
#             order=['LexNSGA2','MLP','Lex','NSGA2','SimAnneal','ElasticNet',  'Random'],palette='Spectral')
# for d,df_g in df_reg.groupby('dataset'):
#     h = plt.figure()
sns.set_style('whitegrid')
sns.boxplot(data=df_sum,y='norm_score',x='method',notch=False,showfliers=False, 
             palette='Spectral',
            order=Order)
#             order=['RF','KernelRidge','','CN','Corr','SXO','CNSXO','CorrSXO'])
#             order=['RF','KernelRidge','MLP'])

# plt.title(d)
# plt.ylim([-1,1])
plt.xticks(rotation=45,size=12)
plt.ylabel('Score (Normalized $R^2$)',size=14)
plt.xlabel('')
# plt.xlabel('Algorithm',size=14)
plt.tight_layout()
plt.savefig('../../paper/figs/boxplot_score.pdf')

print('method','median','mean','std')
for m,df in df_reg.groupby('method'):
    print(m,df['score'].median(),df['score'].mean(),df['score'].std())

In [ ]:
import matplotlib.pyplot as plt
import math
%matplotlib inline
sns.set_style('whitegrid')
# plt.figure(figsize=(20,5))
df_reg['norm_score'] = df_reg.groupby('dataset')['score'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df_sum=pd.DataFrame()#columns=['dataset','method','med_score'])
df_sum['med_score'] = df_reg.groupby(['dataset','method'])['score'].median().drop_duplicates()
df_sum['norm_score'] = df_reg.groupby(['dataset','method'])['norm_score'].median().drop_duplicates()

df_sum['count'] = df_reg.groupby(['dataset','method'])['score'].count()
# print(df_sum[:10])
df_sum=df_sum.reset_index(level=['dataset', 'method'])
# df_sum=df_sum.unstack(level=-1)
# print(df_sum)
# plt.figure()
# sns.barplot(data=df_sum,y='count',x='dataset',hue='method')
# plt.show()
# sns.boxplot(data=df_sum,x='method')
# sns.boxplot(data=df_sum,y='med_score',x='method',notch=True,showfliers=False, 
#             order=['LexNSGA2','MLP','Lex','NSGA2','SimAnneal','ElasticNet',  'Random'],palette='Spectral')

nc = 2
nr = math.ceil(len(df_reg.dataset.unique())/nc)
print(nr,'x',nc)
h = plt.figure(figsize=(7,10))
for i,(d,df_g) in enumerate(df_reg.groupby('dataset')):
    h.add_subplot(nr,nc,i+1)
    g =sns.boxplot(data=df_g,y='score',x='method',notch=False,showfliers=True, 
        order=Order,palette='Spectral')
    plt.title(d)
# plt.ylim([-1,1])
#     if i == len(df_reg.groupby('dataset'))-1:
#         g.legend(loc=(1.01,0.1))
    if (i+1)/nc>nr-1:
        plt.xticks(rotation=45,size=12)
        plt.xlabel('Algorithm',size=14)
    else:
        plt.gca().set_xticks([])
        plt.xlabel('')
    if np.mod(i,nc)==0:
        plt.ylabel('Score ($R^2$)',size=14)
    else:
        plt.ylabel('')
    plt.yticks(size=12)
    if d == 'enh': plt.ylim(0.9974,0.9977)
plt.tight_layout()
plt.savefig('../../paper/figs/boxplot_score_by_dataset.pdf')


In [ ]:
print(df_reg.dtypes)

In [ ]:
# ranks
import matplotlib.pyplot as plt
%matplotlib inline
df_ranks = pd.DataFrame(dtype=float)
# tmp = df_reg.groupby(['dataset','method'])['score']
df_ranks['median_scores'] = df_reg.groupby(['dataset','method'])['score'].median().drop_duplicates()

df_ranks= pd.DataFrame({"rank_r2": df_ranks.groupby('dataset')['median_scores'].rank(ascending=False)}
                      ).reset_index()
# print(df_ranks)
# # plt.figure(figsize=(20,5))
df_ranks.to_csv('tuning_r2_ranks.csv',index=False)
sns.barplot(data=df_ranks,y='rank_r2',x='method',edgecolor=(0,0,0),capsize=0.1,errwidth=1,
            fill=True,palette='Spectral',
            order=Order)
#             order=['RF','KernelRidge','','CN','Corr','SXO','CNSXO','CorrSXO'])
# # plt.ylim([-1,1])
plt.xticks(rotation=45,size=12)
plt.ylabel('Median Score ranks',size=14)
plt.xlabel('')
plt.tight_layout()
# plt.savefig('../paper/figs/barplot_ranks.pdf')

In [ ]:
df_ranks = pd.DataFrame(dtype=float)
df_reg.loc[df_reg['method']=='MLP','size'] = df_reg.loc[df_reg['method']=='MLP']['num_params'] 
df_reg['size'] = df_reg['size'].apply(pd.to_numeric)
df_ranks['median_sizes'] = df_reg.groupby(['dataset','method'])['size'].median()
# print(df_ranks)
# df_ranks.dropna(inplace=True)
df_ranks= pd.DataFrame({"rank_size": df_ranks.groupby('dataset')['median_sizes'].rank(ascending=True)}
                      ).reset_index()
# print(df_ranks)
df_ranks.to_csv('size_ranks.csv',index=False)
sns.barplot(data=df_ranks,y='rank_size',x='method',edgecolor=(0,0,0),capsize=0.1,errwidth=1,
            fill=True,palette='Spectral',
            order=Order)
#             order=['RF','KernelRidge','','CN','Corr','SXO','CNSXO','CorrSXO'])
# # plt.ylim([-1,1])
plt.xticks(rotation=45,size=12)
plt.ylabel('Median size ranks',size=14)
plt.xlabel('')
plt.tight_layout()
plt.savefig('../../paper/figs/barplot_size_ranks.pdf')

In [ ]:
# import matplotlib.pyplot as plt
%matplotlib inline
# plt.figure(figsize=(20,5))
# df_data['real_name'] = [str(did).strip()+'_'+name 
#                         for did,name in zip(df_data['did'].values,df_data['name'].values)]
# size_dict = df_data.set_index('real_name').loc[:,'NumberOfInstances'].to_dict()
# print(size_dict)
# df_reg['dataset_size'] = df_reg['dataset'].apply(lambda x: int(size_dict[x]))
df_reg['num_params'] = df_reg['num_params'].apply(pd.to_numeric)
sns.boxplot(data=df_reg,y='num_params',x='method',notch=True,showfliers=False,palette='Spectral',
             order=Order)
#             order=['RF','KernelRidge','','CN','Corr','SXO','CNSXO','CorrSXO'])
#             order=['RF','KernelRidge','MLP'])

# plt.ylim([-1,1])
plt.xticks(rotation=45,size=12)
plt.ylabel('Size (# parameters)',size=14)
plt.xlabel('')
# plt.gca().set_yscale('log')

plt.tight_layout()
plt.savefig('../../paper/figs/boxplot_params.pdf')


In [ ]:
# import matplotlib.pyplot as plt
%matplotlib inline
pal = sns.color_palette('Spectral',n_colors=len(Order))
print(len(pal))

for i,o in reversed(list(enumerate(Order))):
    if o == 'KernelRidge':
        print(i)
        del pal[i]
# plt.figure(figsize=(20,5))
# df_data['real_name'] = [str(did).strip()+'_'+name 
#                         for did,name in zip(df_data['did'].values,df_data['name'].values)]
# size_dict = df_data.set_index('real_name').loc[:,'NumberOfInstances'].to_dict()
# print(size_dict)
# df_reg['dataset_size'] = df_reg['dataset'].apply(lambda x: int(size_dict[x]))
df_reg.loc[df_reg['method']=='MLP','size'] = df_reg.loc[df_reg['method']=='MLP']['num_params'] 
df_reg['size'] = df_reg['size'].apply(pd.to_numeric)

sns.boxplot(data=df_reg,y='size',x='method',notch=True,showfliers=False,palette='Spectral',
            order=[o for o in Order if o != 'KernelRidge'])
# plt.ylim([-1,1])
plt.xticks(rotation=45,size=12)
plt.ylabel('Size (# nodes)',size=14)
plt.xlabel('')
# plt.gca().set_yscale('log')

plt.tight_layout()
plt.savefig('../../paper/figs/boxplot_size.pdf')

In [ ]:
print([g for g in plt.rcParams.keys() if 'grid' in g])
sns.set_style('whitegrid')
sns.boxplot(data=df_reg,y='time',x='method',notch=True,showfliers=False, palette='Spectral',
            order=Order)
#             order=['','CN','Corr','SXO','CNSXO','CorrSXO'])
plt.ylabel('Time (s)',size=14)
plt.xticks(rotation=45,size=12)
plt.xlabel('')
# plt.gca().set_yscale('log')

plt.tight_layout()
plt.savefig('../../paper/figs/boxplot_time.pdf')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
pal = sns.color_palette('Spectral',n_colors=len(Order))
print(len(pal))

# for i,o in reversed(list(enumerate(Order))):
#     if o not in ['Feat','FeatCorr','FeatCN','MLP','ElasticNet']:
#         print(i)
#         del pal[i]
        
df_reg['norm_corr'] = df_reg.groupby('dataset')['corr'].transform(lambda x: (x - x.min()) / (x.max() - x.min())).apply(lambda x: x if not np.isnan(x) else 1.0)
df_sum=pd.DataFrame()#columns=['dataset','method','med_score'])
df_sum['med_corr'] = df_reg.groupby(['dataset','method'])['corr'].median().drop_duplicates()
df_sum['norm_corr'] = df_reg.groupby(['dataset','method'])['norm_corr'].median().drop_duplicates()
df_sum=df_sum.reset_index(level=['dataset', 'method'])
# sns.set_palette(pal)
h = plt.figure()
df_sum.loc[df_sum.method=='ElasticNet','method'] = 'Baseline'
sns.set(font_scale=1.7)
sns.set_style('whitegrid')
sns.boxplot(data=df_sum,y='med_corr',x='method',notch=False,showfliers=False,
        palette=pal)
# , order=['Feat','FeatCorr','FeatCN','MLP','Baseline'])
#             ,order=[o for o in Order if o not in ['RF']])
#         order=['','SXO','CN','Corr','CNSXO','CorrSXO'])
# plt.title(d)
plt.ylabel('Mean Representation Correlation',size=14)
plt.xlabel('')
plt.xticks(rotation=45,size=12)
plt.tight_layout()
plt.savefig('../../paper/figs/boxplot_corr_tuning.pdf')

In [ ]:
pal = sns.color_palette('Spectral',n_colors=len(Order))
df_reg['norm_cond'] = df_reg.groupby('dataset')['cond'].transform(lambda x: (x - x.min()) / (x.max() - x.min())).apply(lambda x: x if not np.isnan(x) else 1.0)
df_sum=pd.DataFrame()#columns=['dataset','method','med_score'])
df_sum['med_cond'] = df_reg.groupby(['dataset','method'])['cond'].median().drop_duplicates()
df_sum['norm_cond'] = df_reg.groupby(['dataset','method'])['norm_cond'].median().drop_duplicates()
df_ranks = df_sum
df_sum=df_sum.reset_index(level=['dataset', 'method'])

df_ranks= pd.DataFrame({"rank_cond": df_ranks.groupby('dataset')['med_cond'].rank(ascending=False)}
                      ).reset_index()
print(df_sum.columns)
# del pal[2]
# df_tmp = df_reg
# df_tmp['cond'] = df_tmp['cond'].apply(lambda x: x if not np.isinf(x) else 10**20)
h = plt.figure()
sns.boxplot(data=df_sum,y='med_cond',x='method',notch=False,showfliers=False,
        palette='Spectral') #,order=[o for o in Order if o not in ['RF']])
#         order=['','SXO','CN','Corr','CNSXO','CorrSXO'])
# plt.title(d)
plt.ylabel('Representation Condition Number',size=14)
plt.xlabel('')
plt.xticks(rotation=45,size=12)
plt.tight_layout()
# plt.ylim(0,10e12)
# plt.gca().set_yscale('log')

plt.savefig('../../paper/figs/boxplot_cond.pdf')

In [ ]:
pal = sns.color_palette('Spectral',n_colors=7)
del pal[2]
print(pal)
# sns.set_palette(pal)
for d,df_g in df_reg.groupby('dataset'):
    h = plt.figure()
    sns.boxplot(data=df_g,y='corr',x='method',notch=False,showfliers=False,
            palette=sns.color_palette(pal),
                order=Order)
#             order=['','SXO','CN','Corr','CNSXO','CorrSXO'])
    plt.title(d)
    plt.ylabel('Mean Feature Correlation',size=14)
    plt.xlabel('')
    plt.xticks(rotation=45,size=12)
    plt.tight_layout()
# plt.savefig('../paper/figs/boxplot_corr.pdf')

## Hyperparameter results

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import argparse
from glob import glob
import math 

frames = []
# load regression results
# for f in glob('results/regression/*/*cv_results.csv'):
for f in glob(rdir + '*/*cv_results.csv'):
    if 'MLP' in f and 'results_post' not in f:
            continue 
#     print(f)
    if 'Random' not in f:
        try:
            df = pd.read_csv(f,sep=',',index_col=None)
            df['Feat'] = True if 'Feat' in f else False
            df['method'] = f.split('_')[-3]
            df['dataset'] = f.split('/')[-2]
#             if 'Feat' in f:

#                 df['hc_or_bp'] = ['HC' if np.isnan(d) else 'BP' for d in df['param_backprop']]
#                 df.loc[df['param_backprop'].isna(),'param_backprop']=False
#                 df.loc[df['param_hillclimb'].isna(),'param_hillclimb']=False

            frames.append(df)

        except pd.io.parsers.EmptyDataError:
            print(f,'is empty')
df_hp = pd.concat(frames)
df_hp_feat = df_hp[df_hp.Feat==True]
df_hp_feat['method_nice'] = df_hp_feat['method'].apply(lambda x: {'FeatR':'Standard',
                                                                  'FeatRXOR':'ResXO',
                                                                   'FeatSXOR':'StageXO'}[x])
# df_reg.loc[df_reg['method']=='SimAnn'] = 'Sim Anneal'
print(df_hp.columns)
print('methods:',df_hp.method.unique())
print('parameters:',[c for c in df_hp.columns if 'param_' in c])

# Best Parameters

In [ ]:
for i, (m,df_m) in enumerate(df_hp_feat.groupby('method')):
    print(10*'=','\n',m)
    for p in [c for c in df_m.columns if 'param_' in c]:
        df_tmp = df_m.groupby([p,'dataset'])['rank_test_score'].median() 
        med_param_scores = df_tmp.groupby(p).median()
#         med_dataset_scores = df_m.groupby('dataset')['rank_test_score'].median()
        print(p,':',med_param_scores.idxmin(),', rank=',med_param_scores.min())
#         print(p,med_param_scores)
    print(10*'=')
#     idx = med_dataset_scores.idxmin() 
#     print(m,idx)
#     print(m,'best cross rate:',df_m) 

# Cross rate

In [ ]:
g = sns.boxplot(data=df_hp_feat,y='rank_test_score',x='param_cross_rate',notch=True,showfliers=False,
                hue='method_nice',palette='Spectral',hue_order=['Standard','ResXO','StageXO'])
# g.legend(loc=[1.01,0.5])
g.legend(loc='lower left',fontsize=10)
plt.title('all problems')
plt.ylabel('Test Score Rank')
plt.xlabel('Crossover/Mutation Rate')
plt.tight_layout()
plt.savefig('../../paper/figs/boxplot_cross_rate.pdf')
n_datasets = len(df_hp_feat.dataset.unique())
h = plt.figure(figsize=(7,10))
nc = 2
nr = math.ceil(len(df_reg.dataset.unique())/nc)
for i, (d,df_g) in enumerate(df_hp_feat.groupby('dataset')):
    ax = h.add_subplot(nr, nc, i+1)
    g = sns.boxplot(data=df_g,y='mean_test_score',x='param_cross_rate',notch=True,showfliers=False,hue='method'
                ,ax=ax,palette='Spectral')
    plt.title(d)
    if i+1 != len(df_hp_feat.groupby('dataset')):
        g.legend_.remove()
    else:
        plt.legend(loc='lower center',fontsize=10)
    if i+1 not in [7,8]:
        plt.xlabel('')
    if (i+1) % 2 == 0:
        plt.ylabel('')
    else:
        plt.ylabel('Mean CV $R^2$')
plt.tight_layout(pad=0.1, w_pad=0.001, h_pad=0.1)



# root XO rate

In [ ]:

g = sns.boxplot(data=df_hp_feat,y='rank_test_score',x='param_root_xo_rate',notch=True,showfliers=False,
                hue='method_nice')
plt.title('all problems')
g.legend(loc=[1.01,.5])

n_datasets = len(df_hp_feat.dataset.unique())
h = plt.figure(figsize=(7,10))
nc = 2
nr = math.ceil(len(df_reg.dataset.unique())/nc)
for i, (d,df_g) in enumerate(df_hp_feat.groupby('dataset')):
    ax = h.add_subplot(nr, nc, i+1)
    g = sns.boxplot(data=df_g,y='mean_test_score',x='param_root_xo_rate',notch=True,showfliers=False,
                    hue='method_nice', ax=ax,palette='Spectral')
    plt.title(d)
    if i+1 != len(df_hp_feat.groupby('dataset')):
        g.legend_.remove()
    else:
        plt.legend(loc='lower center',fontsize=8)
    if i+1 not in [7,8]:
        plt.xlabel('')
    if (i+1) % 2 == 0:
        plt.ylabel('')
    else:
        plt.ylabel('Mean CV $R^2$')
plt.tight_layout(pad=0.1, w_pad=0.001, h_pad=0.1)

#     plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))

#             order=['Lex','LexNSGA2','NSGA2','SimAnn','Random'])

In [ ]:
df_hp_feat['norm_mean_test_score'] = (df_hp_feat.groupby('dataset')['mean_test_score']
                                      .transform(lambda x: (x - x.min()) / (x.max() - x.min())))

h  = plt.figure()
sns.boxplot(data=df_hp_feat,y='rank_test_score',x='param_root_xo_rate',notch=True,showfliers=False,
            hue='method')

#     plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))

#             order=['Lex','LexNSGA2','NSGA2','SimAnn','Random'])

# Feedback

In [ ]:
sns.boxplot(data=df_hp_feat,y='rank_test_score',x='param_fb',hue='method_nice',
            notch=True,showfliers=False)
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
n_datasets = len(df_hp_feat.dataset.unique())
h = plt.figure(figsize=(7,10))
nc = 2
nr = math.ceil(len(df_reg.dataset.unique())/nc)
for i, (d,df_g) in enumerate(df_hp_feat.groupby('dataset')):
    ax = h.add_subplot(nr, nc, i+1)
    g = sns.boxplot(data=df_g,y='mean_test_score',x='param_fb',notch=True,showfliers=False,hue='method_nice',
                ax=ax,palette='Spectral',hue_order=['Standard','ResXO','StageXO'])
    plt.title(d)
    if i+1 != len(df_hp_feat.groupby('dataset')):
        g.legend_.remove()
    else:
        plt.legend(loc='lower left',fontsize=10)
    if i+1 not in [7,8]:
        plt.xlabel('')
    else:
        plt.xlabel('feedback')
    if (i+1) % 2 == 0:
        plt.ylabel('')
    else:
        plt.ylabel('Mean CV $R^2$')
plt.tight_layout(pad=0.1, w_pad=0.001, h_pad=0.1)
h.savefig('../../paper/figs/boxplot_feedback_by_dataset.pdf')

In [ ]:
sns.boxplot(data=df_hp_feat,y='rank_test_score',x='param_softmax_norm',hue='param_fb',
            notch=True,showfliers=False)
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [ ]:
import math
n_datasets = len(df_hp_feat.dataset.unique())
h = plt.figure(figsize=(18,9))
nr = 
for i, (d,df_g) in enumerate(df_hp_feat.groupby('dataset')):
    ax = h.add_subplot(math.ceil(n_datasets/4), 4, i+1)
#     [df_g.method=='FeatRXOR']
    g=sns.boxplot(data=df_g,y='mean_test_score',x='param_fb',hue='method',notch=False,showfliers=False, 
                ax = ax)
    if i+1 != len(df_hp_feat.groupby('dataset')):
        g.legend_.remove()
    else:
        plt.legend(loc=[1.01,0.5])
    plt.title(d)
plt.tight_layout()

In [ ]:
df_reg.head()
mask = (np.array(df_reg.method=='Feat') & np.array(df_reg.dataset=='690_visualizing_galaxy'))

df_reg.loc[mask,:]

# visualize pairplots of multiple hyperparameters

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
import itertools as it
variables=['param_fb','param_cross_rate','param_root_xo_rate']
norm = cm.colors.Normalize(vmin=df_hp_feat['mean_test_score'].min(),vmax=df_hp_feat['mean_test_score'].max())
df_hp_feat['norm_test_score'] = ((df_hp_feat['mean_test_score'] - df_hp_feat['mean_test_score'].median())/
                (df_hp_feat['mean_test_score'].quantile(q=0.75)-df_hp_feat['mean_test_score'].quantile(q=0.25))
                    )
for x,y in it.permutations(variables,2):
    h = plt.figure()
    ax = h.add_subplot(111, projection='3d')
    ax.scatter(df_hp_feat[x],df_hp_feat[y],df_hp_feat['mean_test_score'],c=df_hp_feat['rank_test_score'],
              cmap=cm.Blues)
    ax.set_xlabel(x)
    ax.set_ylabel(y)
    ax.set_zlabel('mean_test_score')
    ax.set_zlim(0,1)
plt.show()